In [90]:
import requests
from bs4 import BeautifulSoup
import json

from books.modules import goodreads, notion

# Provide the Goodreads URL to capture the metadata for
URL = "https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111"

r = requests.get(URL)
soup = BeautifulSoup(r.text, "html.parser")
script = soup.find('script', {'type': 'application/ld+json'}).text

# Parse the JSON data
data = json.loads(script)

# Extract the metadata for the book from the JSON data
name = data['name']
image = data['image']
book_format = data['bookFormat']
num_pages = data['numberOfPages']
language = data['inLanguage']
isbn = data['isbn']
rating = data['aggregateRating']['ratingValue']
rating_count = data['aggregateRating']['ratingCount']
review_count = data['aggregateRating']['reviewCount']
authors = [author['name'] for author in data['author']]

# Print the metadata for the book
print(f'Name: {name}')
print(f'Image: {image}')
print(f'Format: {book_format}')
print(f'Number of pages: {num_pages}')
print(f'Language: {language}')
print(f'ISBN: {isbn}')
print(f'Rating: {rating}')
print(f'Number of ratings: {rating_count}')
print(f'Number of reviews: {review_count}')
print(f'Authors: {authors}')
import urllib.parse

'.'.join(urllib.parse.urlparse(URL).hostname.split('.')[-2:])

Name: Machine Learning Design Patterns: Solutions to Common Challenges in Data Preparation, Model Building, and MLOps
Image: https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1599583594i/55275019.jpg
Format: Paperback
Number of pages: 400
Language: English
ISBN: 9781098115784
Rating: 4.2
Number of ratings: 146
Number of reviews: 21
Authors: ['Valliappa Lakshmanan', 'Sara Robinson', 'Michael Munn']


In [95]:
# Set the URL you want to check
url = 'https://www.example.com/path/to/page'

# Set the domain you want to check against
domain = 'example.com'

# Parse the URL and extract the domain
parsed_url = urllib.parse.urlparse(url)
url_domain = parsed_url.hostname

# Strip the subdomain from the domain
url_domain_parts = url_domain.split('.')
stripped_url_domain = '.'.join(url_domain_parts[-2:])


# Check if the domain of the URL matches the desired domain
if stripped_url_domain == domain:
    print(f'The URL {url} is from the domain {domain}')
else:
    print(f'The URL {url} is NOT from the domain {domain}')

The URL https://www.example.com/path/to/page is from the domain example.com
